# An example of parameters estimation using first BAO peak

The early universe consisted of a hot, dense plasma of electrons and baryons (protons and neutrons). Photons (light particles) traveling in this universe were essentially trapped, unable to travel for any considerable distance before interacting with the plasma via Thomson scattering. As the universe expanded, the plasma cooled to below 3000 K—a low enough energy such that the electrons and protons in the plasma could combine to form neutral hydrogen atoms. This recombination happened when the universe was around 379,000 years old, or at a redshift of z = 1089. Photons interact to a much lesser degree with neutral matter, and therefore at recombination the universe became transparent to photons, allowing them to decouple from the matter and free-stream through the universe.

Barionic acostic oscillation are standard rulers because all the gravitational wells produce the same radius in the acoustic oscillations.

In general the comoving distance between a source at redshift $z$ and us is: $$\boxed{\chi = c\int_{0}^{z}\frac{d z'}{H(z')}}$$ In a euclidean universe ($k=0$) the angles $\theta$, subtended by a standard ruler of comoving size $l_c$ is $$\boxed{\theta=\frac{l_c}{\chi}}$$

Our standard ruler is the radius of the soundwave until the last scattering, this is also known as the sound horizon. And it is given by $$\boxed{l_c=\int^{\infty}_{z_{ls}}  c_{s}(z)\frac{d z'}{H(z')}}$$

For an adiabatic fluid the the sound velocity is given by $$c_s^2=\frac{\dot{P}}{\dot{\rho}} = \frac{\frac{\dot{\rho_\gamma}}{3}}{\dot{\rho_\gamma}+\dot{\rho_b}} = \left(3+3\frac{\dot{\rho_b}}{\dot{\rho_\gamma}} \right)^{-1}$$ 

from the continuity equation for each component $\dot{\rho}+3H(\rho + p) = 0$ we have $$\frac{\dot{\rho_b}}{\dot{\rho_\gamma}} = \frac{3}{4}\frac{\rho_b}{\rho_{\gamma}} = \frac{3}{4}\frac{\Omega_{b0}a^{-3}}{\Omega_{\gamma 0}a^{-4}} = \frac{3}{4}\frac{\Omega_{b0}}{\Omega_{\gamma 0}(1+z)}$$ Finally, the sound velocity is given by: $$\boxed{ \frac{c_s}{c
}= \left( 3+\frac{9}{4}\frac{\Omega_{b0}}{ \Omega_{\gamma 0}z}  \right)^{-1/2}}$$

where it has been approximated $z \approx z+1$ since we are in the period of last scatering where $z_{ls}=1090$ .
Is important to recall that dark matter does not interact with phothons, so that the sound speed is calculate only for a gas of barions an photons.

In the practice, from measurement we know the $\theta$ from the first peak of the CMB correlation function ($\theta \approx \frac{\pi}{l}$ ) or the first peak of the matter power spectrum. So it is convinient to write explicitly a theoretical function for this subtendent angle, i.e $$\boxed{\Theta = \frac{ \int^{\infty}_{z_{ls}}  c_{s}(z)\frac{d z'}{H(z')}} {c\int_{0}^{z}\frac{d z'}{H(z')}} = \left[ \int^{\infty}_{z_{ls}}  \frac{c_{s}(z)}{c}\frac{d z'}{E(z')} \right]\left[ \int_{0}^{z}\frac{d z'}{E(z')}\right]^{-1}}$$ where $$E(z) = \sqrt{(1-\Omega_{m0}) + \Omega_{m0}(1+z)^{3} + \Omega_{R0}(1+z)^{4} },$$ 

This is just a fancy way to visualize data

In [151]:
def PrettyPrint(data):
    from prettytable import PrettyTable
    x = PrettyTable(data.dtype.names)    
    for row in data:
        x.add_row(row)
    print(x)

In [152]:
def sound_speed(Ob0, Og0,z):
    import numpy as np
    return np.power(3+(9*Ob0)/(4*Og0*z) , -0.5)

In [153]:
def Em1(z, parameters):
    import numpy as np
    E =  np.sqrt( (1 - parameters[0]) + parameters[0] * np.power(1 + z, 3) + parameters[1] * np.power(1 + z, 4 ) )
    return 1./E    

In [154]:
def sound_integrand(z, parameters):
    return sound_speed(parameters[2],parameters[3],z)*Em1(z,parameters)

In [155]:
def sound_horizon(zls, parameters):
    import numpy as np
    from scipy.integrate import quad
    I,e = quad(sound_integrand, zls, np.inf, args=(parameters))
    return I

In [156]:
def comoving_horizon(z,parameters):
    from scipy.integrate import quad
    I,e = quad(Em1, 0, z,  args=(parameters))
    return I

In [157]:
def theta_teo(z,zls, parameters):
    return sound_horizon(zls, parameters)/comoving_horizon(z,parameters)    

And we define our $\chi^{2}$ test respect to this quantity. So we want to find the parameters that minimizes 
$$\chi^{2}= \sum \left(\frac{\Theta_{teo}-\Theta_{exp}}{\sigma_{\Theta}}\right)^{2} $$ 

In [158]:
def chi2(thetat, thetaobs,  sigma):
    import numpy as np
    aux = 0
    for i in range(len(thetat)):
        aux += np.power( (thetat[i] - thetaobs[i]) / sigma[i], 2)
    return aux

In [159]:
def runOm(steps, minval, maxval,  data,  pars):
    import numpy as np
    import pandas as pd
    thetaobs = data['theta'] * math.pi / 180.
    sigma =  data['sigma']
    
    thetateo = [None] * len(thetaobs)
    z_bao = 1060
    #Output data
    keys =  ["Om",  "Or", "Ob0", "Og0", "chi2"]
    outdata = { key : [None] for key in keys }  
    delta =  (maxval - minval) / steps
    
    for j in range(1, steps + 1):
        pars[0] = minval + delta * j
        for k in range(len(data['z'])):
            thetateo[k] =  theta_teo(data['z'][k],z_bao, pars)
        chiaux = chi2(thetateo, thetaobs, sigma)
        outdata["Om"].append(pars[0])
        outdata["Or"].append(pars[1])
        outdata["Ob0"].append(pars[2])
        outdata["Og0"].append(pars[3])
        outdata["chi2"].append(chiaux)
        
    df = pd.DataFrame(outdata)
    #print(df)
    boolean =  (df['chi2'] == np.min(df['chi2']))
    print ('Omega_m =', list(df.loc[boolean, 'Om'])[0] ,  '\n' )
    print ('Omega_r =', list(df.loc[boolean, 'Or'])[0] ,  '\n' )
    print ('Omega_b0 =', list(df.loc[boolean, 'Ob0'])[0] ,  '\n' )
    print ('Omega_g0 =', list(df.loc[boolean, 'Og0'])[0] ,  '\n' )
    print ('chi2 =', list(df.loc[boolean, 'chi2'])[0] ,  '\n' )
    print ('reduced_chi2 =', list(df.loc[boolean, 'chi2'])[0] / (len(data) - 1) ,  '\n' )

In [160]:
def runOm_cmbpoint(steps, minval, maxval,  data,  pars):
    import numpy as np
    import pandas as pd
    thetaobs = data['theta'] * math.pi / 180.
    sigma =  data['sigma']
    
    #increase data table
    thetaobs=np.append(thetaobs,math.pi/302.)
    sigma=np.append(sigma, math.pi*0.2/(302.**2))
    dsize=len(thetaobs)
    
    thetateo = [None] * len(thetaobs)
    z_bao = 1060
    #Output data
    keys =  ["Om",  "Or", "Ob0", "Og0", "chi2"]
    outdata = { key : [None] for key in keys }  
    delta =  (maxval - minval) / steps
    
    for j in range(1, steps + 1):
        pars[0] = minval + delta * j
        for k in range(dsize):
            if (k==(dsize-1)):
                thetateo[k] =  theta_teo(1090,1090, pars)
            else:    
                thetateo[k] =  theta_teo(data['z'][k],z_bao, pars)
        chiaux = chi2(thetateo, thetaobs, sigma)
        outdata["Om"].append(pars[0])
        outdata["Or"].append(pars[1])
        outdata["Ob0"].append(pars[2])
        outdata["Og0"].append(pars[3])
        outdata["chi2"].append(chiaux)
        
    df = pd.DataFrame(outdata)
    #print(df)
    boolean =  (df['chi2'] == np.min(df['chi2']))
    print ('Omega_m =', list(df.loc[boolean, 'Om'])[0] ,  '\n' )
    print ('Omega_r =', list(df.loc[boolean, 'Or'])[0] ,  '\n' )
    print ('Omega_b0 =', list(df.loc[boolean, 'Ob0'])[0] ,  '\n' )
    print ('Omega_g0 =', list(df.loc[boolean, 'Og0'])[0] ,  '\n' )
    print ('chi2 =', list(df.loc[boolean, 'chi2'])[0] ,  '\n' )
    print ('reduced_chi2 =', list(df.loc[boolean, 'chi2'])[0] / (len(data) - 1) ,  '\n' )

In [161]:
import numpy as np
import itertools as it
import pandas as pd
import math
data = np.loadtxt("theta_bao.dat",  dtype={'names': ('z', 'theta' ,'sigma'), 'formats': ('f4', 'f4', 'f4')})

parameters =  [None,8.4*(10**-5),0.05,5*(10**-5)]

runOm(100, 0, 1,  data,  parameters)

Omega_m = 0.21 

Omega_r = 8.400000000000001e-05 

Omega_b0 = 0.05 

Omega_g0 = 5e-05 

chi2 = 0.004577090566807889 

reduced_chi2 = 0.00035208388975445295 



In [162]:
runOm_cmbpoint(100, 0, 1,  data,  parameters)

Omega_m = 0.29 

Omega_r = 8.400000000000001e-05 

Omega_b0 = 0.05 

Omega_g0 = 5e-05 

chi2 = 0.6019105767425019 

reduced_chi2 = 0.04630081359557707 

